# Прикладная статистика. ДЗ 1.
# Академия Аналитиков Авито

__Правила:__
- Финальный дедлайн: **2022-11-30 23:59**. 
- После того как ваше решение проверят и ответят, вам дается неделя на исправление тех задач, о которых скажет проверяющий. Ответ и обсуждение решения — в телеграме.

- Выполненную работу нужно отправить
    - в чатик HW1-<ваше имя> через бота @AAA_stats23_bot
    - или в личные сообщения боту.
- В качестве решения нужно отправить файл ipynb. Ссылка на интернет-ресурсы не принимается. Не публикуйте решения в открытом доступе!
- Для выполнения задания используйте этот ноутбук в качествие основы, ничего не удаляя из него. **При этом можно добавлять новые ячейки!**
- в ячейках с комменарием `#Автопроверка` нужно заполнить содержимое функций и классов (если есть), которые будут уже объявлены в этой ячейке. При этом:
    - Нельзя убрирать или переставять `#Автопроверка` в ячейке. 
    - Нельзя менять сигнатуру и возвращаемое значение функций. То есть добавлять любой код можно, но удалять, что уже написано - нельзя.
    - Нельзя ничего импортировать в таких ячейках. Все доступные для использования библиотеки будут указаны заранее. Такие слова, как `import`, `globals`, `locals`, `eval`, `exec` также нельзя использовать внутри ячеек.
    - Нельзя использовать библиотеки, кроме тех, что указаны в задании. Ваш код должен работать именно с эти набором библиотек без любого дополнительного импорта!
    - Нельзя использовать код из других ячеек ноутбука (кроме ячейки с импортом, в которой указаны все доступные библиотеки). Единственное исключение - если вы проставите в начало такой ячейки слово `#Автопроверка`. Тогда вы можете использовать код из этой ячейки.
    - В случае нарушения этого правила автопроверка будет провалена и вы не получите часть баллов за задачу. 
    - В случае, если есть несколько ячеек автопроверки, то в каждой такой ячейке можно использовать созданные вами функции (или классы) из других ячеек автопроверки.

In [1]:
from collections import namedtuple
from scipy.stats import binom
import math
import numpy as np

# Общие замечания по задачам с автопроверкой

Во всех задачах далее, где потребуется автопроверка, можно использовать только следующие библиотеки:

```
from collections import namedtuple
from scipy.stats import binom
import math
import numpy as np
```

Еще раз обращаем внимание, что в ячейках автопроверки __не__ должны быть импортированы какие-либо библиотеки. В других ячейках импортировать библиотеки можно, но при проверке использованы будут только указанные.

## Задача 1. 3 балла

[Осьминог Пауль](https://www.championat.com/football/article-3238881-samye-izvestnye-zhivotnye-predskazateli.html) 14 раз пробовал угадать победителя футбольного матча.
12 раз он угадал правильно, 2 раза — ошибся. Мы хотим проверить гипотезу:

 - $H_0$: осьминог угадывает победителя матча с вероятностью 0.5
 - $H_1$: осьминог выбирает победителя  матча с вероятностью $\neq$ 0.5

#### Пункт А. 1 балл: 

### На зачет

Вам нужно разработать статистический критерий для проверки этой гипотезы, а также посчитать p-value.

#### Теоретические выкладки


---

$\xi_i = 1$ если осьминог угадал, $0$ иначе 

$\xi_i \sim$ Bernoulli($\mu$)

Таким образом наши гипотезы:

 - $H_0$: $\mu = 0.5$
 - $H_1$: $\mu != 0.5$

Наша статистика: $T = \sum \xi_i \overset{H_0}{\sim} Binom(14, 0.5)$

Когда мы отвергаем гипотезу?

 - Пусть реализация статистики $T(\xi^n) = t$. Тода если $P_{H_0}(T(\xi^n) \geq t) \leq \alpha/2$ или $P_{H_0}(T(\xi^n) \leq t) \leq \alpha/2$, то мы отвергаем гипотезу
 $$
        \begin{align}
            &P_{H_0}(T(\xi^n) \geq t) \leq \alpha/2 \Leftrightarrow\\
            &1 - P_{H_0}(T(\xi^n) < t) \leq \alpha/2 \Leftrightarrow\\
            & P_{H_0}(T(\xi^n) < t) = P_{H_0}(T(\xi^n) \leq t - 1) \geq 1 - \alpha/2 \Leftrightarrow\\
            &F_{\text{Binom}(n, p_0)}(t - 1) \geq 1 - \alpha/2 \Leftrightarrow\\
            &t \geq F^{-1}(1 - \alpha/2) + 1
        \end{align}
 $$
 
Аналогично находим левое критическое значение для двустороннего критерия.
  - `critical_value_right = sps.binom(p=p0, n=n).ppf(1 - alpha/2) + 1`
  - `critical_value_left = sps.binom(p=p0, n=n).ppf(alpha/2) - 1`

На языке p-value:
 - p-value есть вероятность при $H_0$ получить настолько же или более экстремальное значение статистики T, чем то, что мы получили, то есть 
 
 `p-value = (1 - binom.cdf(k - 1, n=N, p=p0)) * 2`
 
 Критерий на основе p-value: если p-value $\leq$ $\alpha$, то $H_0$ отвергается, иначе не отвергается.
 
---

Реализуйте критерий `check_paul_criterion(n, k, alpha)`, где 
- `n` &mdash; количество матчей;
- `k` &mdash; количество верных предсказаний от осьминога;
- `alpha` &mdash; уровень значимости критерия.

Функция должна вернуть `PaulCheckResults` с полями
- is_rejected: отверглась или нет гипотеза H_0 на уровне значимости alpha
- pvalue


In [4]:
# Автопроверка

PaulCheckResults = namedtuple('PaulCheckResults', ['is_rejected', 'pvalue'])

def check_paul_criterion(n: int, k: int, alpha: float = 0.05):
    """
    Параметры:
    - n: количество матчей
    - k: количество верных предсказаний от осьминога
    - alpha: уровень значимости критерия.
        
    Возвращает:
    - PaulCheckResults с полями:
        - is_rejected: bool
            - отверглась или нет гипотеза H_0 
                       на уровне значимости alpha
        - pvalue: float
    """

    is_rejected = None
    pvalue = None

    pvalue_onesided = 1 - binom.cdf(k - 1, n=n, p=0.5)
    
    if pvalue_onesided > 0.5:
        pvalue_onesided = binom.cdf(k, n=n, p=0.5)
    
    pvalue = pvalue_onesided * 2
    
    if pvalue <= alpha:
        is_rejected = True
    else:
        is_rejected = False

    return PaulCheckResults(is_rejected, pvalue)


#### Пункт B. 2 балла: 

Какие выводы можно сделать из полученного результата?

In [13]:
results= check_paul_criterion(14, 12)
critical_value_right = binom.ppf(q=1 - 0.05, n=14, p=0.5) + 1
critical_value_left = binom.ppf(q=0.05, n=14, p=0.5) - 1

print(f'Results: {results}')
print(f'Right critical value: {critical_value_right}')
print(f'Left critical value: {critical_value_left}')

Results: PaulCheckResults(is_rejected=True, pvalue=0.012939453125)
Right critical value: 11.0
Left critical value: 3.0


Исходя из результатов, гипотеза $H_0$ отвергается. То есть осьминог не просто угадывает победителя. Гипотезу о том, что осьминог "что-то знает" мы не отвергаем, на уровне значимости $\alpha$.

В природе, конечно, таких магических осьминогов не бывает. На вопрос, что могло бы пойти не так, почему же мы все-таки не отвергаем? 14 измерений может быть мало для ответа на этот вопрос. Может ему в данном случае действительно повезло, и мы попали в те самые 5%. Также эксперимент над осьминогом - странная вещь, он не является субъектом и не может управлять своими действиями - вот, что известно науке. Да и сам эксперимент был поставлен под сомнения, так как дрессировщики открывали кормушки не одновременно, осьминог "указывал" или "шел" на определенные цвета. Вопрос бы решили дальнейшие его предсказания - расширить выборку, по закону больших чисел, среднее бы сошлось к матожиданию.

Я же склоняюсь к версии, что честно предсказал осьминог только 2 первых. Потом букмекеры всбунтовали, и попросили показывать, что предсказал осьминог, после события. Но тогда становится понятным, что снимались 2 сюжета. 

## Задача 2. 3 балла

### На зачет

Мы разработали новый дизайн нашего продукта. Вероятность, что он понравится случайному человеку — $p$, и она нам неизвестна. Мы хотим
проверить $H_0: p = 1$ с помощью статистического критерия c уровнем значимости $\alpha$. Предложить критерий для решения этой задачи.

---
Сформулируем гипотезы:

 - $H_0$: p = 1
 - $H_1$: p < 1
 
Пронализируем каков должен быть наш критерий. Можно понять, что если в нашей выборке в эксперименте попался человек, которому не нравится дизайн, то критерий сразу должен отвергать гипотезу $H_0$, так как $p$, очевидно, неравно 1. 

На этом и основан наш критерий: "Отвергаем H_0, если хотя бы одно значение $X_i \neq 1$", где $X_i \in Bernulli(p)$ 

Введем статистику: $T(X) = \sum X_i$, $T(X) {\sim} Binom(n, p)$

Тогда критерий на основе этой статистики:

"Отвергаем $H_0$, если $T(X) \neq n$"

Таким образом, как можно расчитать мощность критерия: 

$power = 1 - \mathbb{P}_{H_1}(T = n)$, где повторюсь $T(X) {\sim} Binom(n, p)$

или `power = 1 - binom.pmf(n, n=n, p=p)`

Таким образом, с ростом $n$ у нас будет уменьшаться ошибка второго рода, то есть $\mathbb{P}_{H_1}(T = n)$, и тогда мы будем ростить выборку до момента, когда наша экспериментальная мощность станет больше или равна заявленной.

Кстати, $\alpha$ в данном случае будет нулевой, так как вероятность того, что при верности нулевой гипотезы (то есть $p=1$) у нас возникла какая-то $X_i \neq 1$, просто напросто нулевая.


Напишите функцию `calculate_number_of_users(alpha, beta, p)` — скольки людям нужно показать этот дизайн, чтобы добиться мощности `1 - beta` при заданном `p` и уровне значимости `alpha`.

P.S. Утверждается, что на наших тестах ответ не будет превосходить 1000 человек.

In [20]:
#Автопроверка

def calculate_number_of_users(alpha: float, beta: float, p: float):
    """
    Параметры:
    - alpha: уровень значимости
    - beta: инвертированная мощность критерия. мощность = 1 - beta.
    - p: истинная вероятность того, что пользователю понравится дизайн.
    Возвращает:
    - number_of_users: int
        - количество людей, которым надо показать дизайн.
    """
    number_of_users = 0
    current_power = 0
    expectable_power = 1 - beta
    mu0 = 1
    mu = p

    while current_power < expectable_power:
        number_of_users += 1
        current_power = 1 - binom.pmf(number_of_users, n=number_of_users, p=mu)
        if number_of_users % 2 == 0:
            print(f'{number_of_users} users || power: {current_power}')
    return number_of_users

In [21]:
alpha = 0
beta = 0.1
p = 0.99

n = calculate_number_of_users(alpha, beta, p)
power = 1 - binom.pmf(n, n=n, p=p)
print('-'*75)
print(f'{n} users requiered for power = {power}\n')

20 users || power: 0.18209306240276923
22 users || power: 0.19836941046095413
24 users || power: 0.21432185919278124
26 users || power: 0.2299568541948449
28 users || power: 0.24528071279636743
30 users || power: 0.2602996266117198
32 users || power: 0.2750196640421466
34 users || power: 0.2894467727277079
36 users || power: 0.30358678195042654
38 users || power: 0.31744540498961304
40 users || power: 0.3310282414303197
42 users || power: 0.3443407794258564
44 users || power: 0.3573883979152819
46 users || power: 0.37017636879676774
48 users || power: 0.38270985905771204
50 users || power: 0.39499393286246365
52 users || power: 0.4070335535985006
54 users || power: 0.41883358588189046
56 users || power: 0.4303987975228408
58 users || power: 0.44173386145213633
60 users || power: 0.4528433576092388
62 users || power: 0.463731774792815
64 users || power: 0.474403512474438
66 users || power: 0.4848628825761967
68 users || power: 0.49511411121293036
70 users || power: 0.505161340399793
72 

## Задача 3.

По недостоверной информации (вероятность, что она верна, считаем за 1%), в новой версии нашего сайта есть сложнодетектируемый баг. Мы могли бы попросить разработку его отыскать и починить, но на это уйдет много ресурсов.

К счастью, у нас есть старый AB тест (новая версия сайта vs старая), который мы можем проанализировать и с некоторой вероятностью обнаружить наличие бага просто сравнением выборок. У нас есть три критерия для проверки гипотезы "$H_0$: баги нет, $H_1$: баг есть":
- критерий `A`: $\alpha = 0.02, 1-\beta = 0.50$
- критерий `B`: $\alpha = 0.05, 1-\beta = 0.60$
- критерий `C`: $\alpha = 0.10, 1-\beta = 0.70$

Если критерий находит баг, мы просим разработчиков потратить силы и починить. На это у них уйдет усилий на 1 М ₽ независимо от того, найдут они баг или нет.
Если критерий не найдет баг, затраты разработчиков будут нулевыми, но из-за бага мы потеряем в конечном итоге 50 М ₽.

#### Пункт А. 2 балла: 

Какой критерий стоит выбрать?

---
Давайте оценим средние потери в каждом случае.

У нас есть события $(H_i, H_j)$, где $H_i$ - гипотеза, которую выбрал критерий, а $H_j$ - гипотеза, которая отображает истину.

* В событии $(H_0, H_0)$ мы теряем 0 млн. руб.
* В событии $(H_0, H_1)$ мы теряем 50 млн. руб. (Вероятность этого события есть $\beta$)
* В событии $(H_1, H_0)$ мы теряем 1 млн. руб. (Вероятность этого события есть $\alpha$)
* В событии $(H_1, H_1)$ мы теряем 1 млн. руб. (Вероятность этого события есть $1-\beta$)

Осталось учесть, что информация не 100% достоверная. Можно разложить вероятности по формуле полной вероятности.

Для этого я хочу посчитать функции риска и минимизировать общий байесовский риск. (Его не было, но я проходил это в ВУЗовской статистике):

Риск для ситуации, когда бага нет: $R_1 = 0 * (1 - \alpha) + 1 * \alpha$

Риск для ситуации, когда бага есть: $R_2 = 50 * \beta + 1 * (1 - \beta)$

Общий байесовский риск: $L = R_1 * 0.99 + R_2 * 0.01 = \alpha * 0.99 + (1 + 49\beta) * 0.01 = 0.99\alpha + 0.49\beta + 0.01$

Таким образом, нам надо минимизировать байесовский риск - посчитаем его для каждого и скажем, для которого критерия он меньше. 

In [17]:
# Критерий А alpha=0.02, 1−beta=0.50
alpha = 0.02
beta = 0.5
L = 0.99 * alpha + 0.49 * beta + 0.01
print(f'Байесвоский риск: {L}')

Байесвоский риск: 0.2748


In [18]:
# Критерий B alpha=0.05, 1−beta=0.60
alpha = 0.05
beta = 0.4
L = 0.99 * alpha + 0.49 * beta + 0.01
print(f'Байесвоский риск: {L}')

Байесвоский риск: 0.2555


In [19]:
# Критерий C alpha=0.10, 1−beta=0.70
alpha = 0.10
beta = 0.3
L = 0.99 * alpha + 0.49 * beta + 0.01
print(f'Байесвоский риск: {L}')

Байесвоский риск: 0.256


Видим, что меньше всего у критерия B.

#### Пункт B. 2 балла: 
Предложите оптимальную стратегию, если потери от ненайденного бага составят вместо 50М:
- 20М рублей;
- 3М рублей;
- 300М рублей.

Добавим еще синтетических критериев, когда мы сидим сложа руки и забиваем на эту информацию (критерий `D`), то есть когда мы говорим гипотезу $H_0$ в любом случае, и когда мы сломя голову бежим искать баг, забивая на эксперимент (критерий `E`), то есть когда мы говорим гипотезу $H_1$ в любом случае. Напишем для них $\alpha$ и $\beta$  

- критерий `D`: $\alpha = 0, \beta = 1$
- критерий `E`: $\alpha = 1, \beta = 0$

Посчитаем для каждого риск опять же c уже добавленными критериями

In [43]:
# 20 млн руб.
# L = alpha*0.99 + (20*beta+ 1 - beta)*0.01 = 0.99*alpha + 0.19*beta + 0.01

def get_L(alpha, beta):
    return 0.99*alpha + 0.19*beta + 0.01

errors = {'A': [0.02, 0.5], 'B': [0.05, 0.4], 'C': [0.1, 0.3], 'D': [0, 1], 'E': [1, 0]}

for criteria, error in errors.items():
    print(f"Байесвоский риск для {criteria}: {get_L(*error)}")

Байесвоский риск для A: 0.1248
Байесвоский риск для B: 0.1355
Байесвоский риск для C: 0.166
Байесвоский риск для D: 0.2
Байесвоский риск для E: 1.0


В случае, если потери составят 20 млн руб. оптимально, выбрать критерий `A`.


In [41]:
# 3 млн руб.
# L = alpha*0.99 + (3*beta + 1 - beta)*0.01 = 0.99*alpha + 0.02*beta + 0.01

def get_L(alpha, beta):
    return 0.99*alpha + 0.02*beta + 0.01

for criteria, error in errors.items():
    print(f"Байесвоский риск для {criteria}: {get_L(*error)}")

Байесвоский риск для A: 0.0398
Байесвоский риск для B: 0.0675
Байесвоский риск для C: 0.115
Байесвоский риск для D: 0.03
Байесвоский риск для E: 1.0


В случае, если потери составят 3 млн руб. оптимально выбрать критерий `D`, то есть ничего не делать.

In [42]:
# 300 млн руб.
# L = alpha*0.99 + (300*beta+ 1 - beta)*0.01 = 0.99*alpha + 2.99*beta + 0.01

def get_L(alpha, beta):
    return 0.99*alpha + 2.99*beta + 0.01

for criteria, error in errors.items():
    print(f"Байесвоский риск для {criteria}: {get_L(*error)}")

Байесвоский риск для A: 1.5248000000000002
Байесвоский риск для B: 1.2555000000000003
Байесвоский риск для C: 1.006
Байесвоский риск для D: 3.0
Байесвоский риск для E: 1.0


В случае, если потери составят 300 млн руб. оптимально выбрать критерий `E`, то есть сломя голову бежать и искать баг. Вполне логично, что критерий C стремится к этому значению, так как он имеет большую мощность и $\alpha$, то есть в бОльших случаях говорит, что баг есть.